In [47]:
import requests
import json
import psycopg2
import pandas as pd 
import sqlalchemy as salc
import numpy as np
import warnings
import datetime
import json
from credentials import * 
warnings.filterwarnings('ignore')

with open("config.json") as config_file:
    config = json.load(config_file)

# Database Connection

In [48]:
database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

# Retreiving repos given GitHub ID

In [49]:
github_id = input("GitHub ID")

QUERY = """
select distinct repo_id from 
(
select contributors.cntrb_id, pull_requests.pr_augur_contributor_id, pull_requests.repo_id 
from contributors, pull_requests
where contributors.cntrb_id=pull_requests.pr_augur_contributor_id
and gh_login='%s') a 

""" % github_id

repo_id = pd.read_sql(QUERY, con=engine)
repo_list = repo_id['repo_id'].tolist()



GitHub ID dpose


In [50]:
if len(repo_list) == 0:
    display("There is no repo_id for the provided GitHub User")
else:
    placeholders= ', '.join(['%s']*len(repo_list))  # "%s, %s, %s, ... %s"
    SQL = """
    SELECT
        programming_language,
        COUNT ( * ) AS language_files 
    FROM
        repo_labor 
    WHERE
        repo_id IN ({}) 
    GROUP BY
        repo_id,
        programming_language 
    ORDER BY
        language_files DESC;
    """.format(placeholders)

    SQL = SQL % tuple(repo_list)

    language_list = pd.read_sql(SQL, con=engine, params=repo_list)
    languages = language_list.head(2)['programming_language'].tolist()
    display(languages)

['JavaScript', 'TypeScript']

# API HEADERS

In [51]:
url = "https://job-search4.p.rapidapi.com/simplyhired/search"
headers = {
    'x-rapidapi-key': credentials['x-rapidapi-key'],
    'x-rapidapi-host': credentials['x-rapidapi-host']
}
trend_dict = dict()

# API REQUEST

In [52]:
job_listing_search = languages[0] + " Language"

if job_listing_search in trend_dict:
    trend_dict[job_listing_search] += 1
else:
    trend_dict[job_listing_search] = 1

querystring = {"query": job_listing_search, "page": "1"}

response = requests.request("GET", url, headers=headers, params=querystring)
result = json.loads(response.text)


job_listing_results = result['jobs']
print("The search returned " + str(len(job_listing_results))  + " results") 


The search returned 19 results


In [9]:
for each_job in job_listing_results:
    print("Job Title: "  + each_job["title"])
    print("Company Name: "  + each_job["company_name"])
    print("Location: "  + each_job["location"])
    print("Description: "  + each_job["description"])
    print("Date posted: "  + each_job["date_posted"])
    print("url : "  + each_job["detail_url"])
    print('\n---------------------------------------------------------------------------------------------------\n')


Job Title: Onsite Web Developer with working SEO knowledge
Company Name: The American Fence Company
Location: Omaha, NE
Description: Experience working with coding languages such as HTML, JQuery, JSON, and Ajax. You will be assisting with website builds, implementing new design features, and…
Date posted: 2021-04-21T00:00:00
url : https://www.simplyhired.com/job/78K7CAiJAmEKv35pHx5y75sxf5PMw5HHXUW9bC7GHO6S6p_lSdtG_g?q=JSON+Language

---------------------------------------------------------------------------------------------------

Job Title: Baseball Systems Developer
Company Name: Arizona Diamondbacks
Location: Phoenix, AZ
Description: Minimum of one year(s) of professional experience developing enterprise web applications in Java or another object-oriented language.
Date posted: 2021-04-30T00:00:00
url : https://www.simplyhired.com/job/Uol78wC15s-zgAP_oklXlMdITY-03PdqDS-3eChhTn2V5L5kET9UaQ?q=JSON+Language

-----------------------------------------------------------------------------

In [22]:
trend_dict

{'JSON Language': 1}